In [18]:
from torch.utils.data import Dataset
import chess
import gc
import torch
#remove .nn if this shits the bed
import torch.nn as nn
import torch.nn.functional as F
import regex as re
import numpy as np
import pandas as pd

# Change this for the minimum elo each game should be
min_elo = 2000

# Change this higher if you want to filter longer games, and lower if you want shorter games
min_game_length = 20

# How large the training set should be
training_set_len = 40_000

# how many layers the NN should have
hidden_layers = 4

# what the size of the NN should be
hidden_size = 200


def board_2_rep(board):
    pieces = ['p', 'r', 'n', 'b', 'q', 'k']
    layers = []
    for piece in pieces:
        layers.append(create_rep_layer(board, piece))
    board_rep = np.stack(layers)
    return board_rep

def create_rep_layer(board, type):
    s = str(board)
    s = re.sub(f'[^{type}{type.upper()} \n]', '.', s)
    s = re.sub(f'{type}', '-1', s)
    s = re.sub(f'{type.upper()}', '1', s)
    s = re.sub(f'\.', '0', s) 

    board_mat = []
    for row in s.split('\n'):
        row = row.split(' ')
        row = [int(x) for x in row]
        board_mat.apprend(row)

    return np.array(board_mat)

def move_2_rep(move, board):
    board.push_san(move).uci()
    move = str(board.pop())

    from_output_layer = np.zeros((8,8))
    from_row = 8 - int(move[1])
    from_column = letter_2_num[move[0]]
    from_output_layer[from_row, from_column] = 1

    to_output_layer = np.zeros((8,8))
    to_row = 8 - int(move[3])
    tow_column = letter_2_num[move[2]]
    to_output_layer[to_row, tow_column] = 1

    return np.stack([from_output_layer, to_output_layer])

def create_move_list(s):
    return re.sub('\d*\. ', ' ', s)

def check_mate_single(board):
    board = board.copy()
    legal_moves = list(board.legal_moves)
    for move in legal_moves:
        board.push_uci(str(move))
        if board.is_checkmate():
            move = board.pop()
            return move
        # add in finding mate in 2
        #_ = board.pop()

def distribution_over_moves(vals):
    probs = np.array(vals)
    probs = np.exp(probs)
    probs = probs / probs.sum()
    probs = probs ** 3
    probs = probs / probs.sum()
    return probs

def choose_move(board, player, color):
    legal_moves = list(board.legal_moves)

    move = check_mate_single(board)
    if move is not None:
        return move
    
    x = torch.Tensor(board_2_rep(board)).float().to('cuda')
    if color == chess.Black: 
        x *= -1
    x = x.unsqueeze(0)
    move = predict(x)
    
    vals = []
    froms = [str(legal_move)[:2] for legal_move in legal_moves]
    froms = list(set(froms))
    for from_ in froms:
        val = move[0,:,:][8 - int(from_[1]), letter_2_num(from_[0])]
        vals.append(val)
    
    probs = distribution_over_moves(vals)

    choosen_from = str(np.random.choice(froms, size = 1, p=probs)[0])[:2]

    vals = []
    for legal_move in legal_moves:
        from_ = str(legal_move)[:2]
        if from_ == choosen_from: 
            to = str(legal_move)[2:]
            val = move[1,:, :][8-int(to[1]), letter_2_num[to[0]]]

    chosen_move = legal_moves[np.argmax(vals)]

    return chosen_move

def predict(x):
    return x.max(dim=1)

class ChessDataset(Dataset):

    def __init__(self, games):
        super(ChessDataset, self).__init__()
        self.games = games
    
    def __len__(self):
        return training_set_len
    
    # selects a random move from a random game
    # don't need to keep track of moves in each game, makes the code simpler, and the dataset loading time much quicker
    def __getitem__(self, index):
        game_i = np.random.randint
        random_game = chess_data['AN'].values[game_i]
        moves = create_move_list(random_game)
        game_state_i = np.random.randint(len(moves) - 1)
        next_move = moves[game_state_i]
        moves = moves[:game_state_i]
        board = chess.Board()

        for move in moves:
            board.push_san(move)
        # matrix representations
        x = board_2_rep(board)
        y = move_2_rep(next_move, board)

        # if its black turn to play, multiply by board matrix by -1
        # CNN always knows that it needs to play the pieces that are represented by positive values
        # if this shits the bed change it to 0
        if game_state_i % 2 == 1:
            x *= -1
        
        return x,y


class module(nn.Module):
    
    def __init__(self, hidden_layers, hidden_size):
        super(PolicyNet, self).__init__()
        self.hidden_layers = hidden_layers
        self.input_layer = nn.Conv2d(6, hidden_size, )
        self.module_list = nn.ModuleList([module(hidden_size) for i in range(hidden_layers)])
        self.output_layer = nn.Conv2d(hidden_size, 2, 3, stride=1, padding=1)

    def forward(self, x):
        x = self.input_layer(x)
        x = F.relu(x)

class PolicyNet(nn.Module):
    def __init__(self):
      super(PolicyNet, self).__init__()
      # uh change these values if needed
      self.conv1 = nn.Conv2d(1, 32, 3, 1)
      self.conv2 = nn.Conv2d(32, 64, 3, 1)
      self.dropout1 = nn.Dropout2d(0.25)
      self.dropout2 = nn.Dropout2d(0.5)
      self.fc1 = nn.Linear(9216, 128)
      self.fc2 = nn.Linear(128, 10)

    # x represents our data
    def forward(self, x):
      # Pass data through conv1
      x = self.conv1(x)
      # Use the rectified-linear activation function over x
      x = F.relu(x)

      x = self.conv2(x)
      x = F.relu(x)

      # Run max pooling over x
      x = F.max_pool2d(x, 2)
      # Pass data through dropout1
      x = self.dropout1(x)
      # Flatten x with start_dim=1
      x = torch.flatten(x, 1)
      # Pass data through ``fc1``
      x = self.fc1(x)
      x = F.relu(x)
      x = self.dropout2(x)
      x = self.fc2(x)

      # Apply softmax to x
      output = F.log_softmax(x, dim=1)
      return output

letter_2_num = {'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6, 'h': 7}
num_2_letter = {0: 'a', 1: 'b', 2: 'c', 3: 'd', 4:'e', 5: 'f', 6:'g', 7: 'h'}

chess_data_raw = pd.read_csv('archive/chess_games.csv', usecols = ['AN', "WhiteElo"])
chess_data = chess_data_raw[chess_data_raw['WhiteElo'] > min_elo]  
del chess_data_raw
gc.collect()
chess_data = chess_data[['AN']]
chess_data = chess_data[~chess_data['AN'].str.contains('{')]
# makes sure these games have like actual length
chess_data = chess_data[chess_data['AN'].str.len() > min_game_length]
print(chess_data.shape[0])

data_train = ChessDataset(chess_data['AN'])
# shuffles the data set, not necessary since its already randomly trained
data_train_loader = torch.utils.data.DataLoader(data_train, batch_size = 32, shuffle=True, drop_last=True)

# metric_from = nn.CrossEntropyLoss()
# metric_to = nn.CrossEntropyLoss()

# loss_from = metric_from(output[:, 0, :], y[:, 0, :])
# loss_to = metric_to(output[:, 1, :], y[:, 1, :])
# loss = loss_from + loss_to

# Create a new chess board
board = chess.Board()

# Play a game
while not board.is_checkmate() and not board.is_stalemate():
    # Print the board
    print(board)

    # Get the current player
    player = board.turn

    # If it's the AI's turn
    if player == chess.WHITE:  # Assuming the AI plays as black
        # Get the AI's move
        move = choose_move(board, player, color=chess.BLACK)
        # Make the move
        board.push(move)
    else:
        # Get the human's move
        move = input("Enter your move: ")
        # Convert the move to a chess.Move object
        move = chess.Move.from_uci(move)
        # Make the move
        board.push(move)

# Print the final board
print(board)

883376
r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
P P P P P P P P
R N B Q K B N R


IllegalMoveError: illegal uci: 'g1f3' in rnbqkbnr/pppppppp/8/8/8/7N/PPPPPPPP/RNBQKB1R b KQkq - 1 1